# Exercise 18

## Analyze class homeworks


In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt

data = pd.read_excel('E18.xlsx')
data.head()

,Sexo,T1,T2,T3,T4,T5,T6
0,H,Un científico de datos y un ingeniero de datos...,NaN,Los árboles de decisión es uno de los algoritm...,NaN,"En el documento ""Do We Need Hundreds of Classi...",Boosting builds models from individual so call...
1,H,Aunque los dos perfiles cuentan con habilidade...,Aunque el conceso no es total en el tema de ma...,NaN,En Machine Learing el propósito de los métodos...,El paper publicado en 2014 hace una evaluación...,Gradient Boosting ClassifierGradient Boosting ...
2,H,Python vs Stata. Analizar información es un pr...,En el marco del Machine Learning encontramos c...,Árbol de clasificación Como su nombre lo indic...,Este es un mecanismo que utiliza varios algori...,El paper analiza 179 clasificadores de 17 dife...,Gradient Boosting Classifier busca aprender de...
3,M,Los algoritmos de machine learning buscan pred...,Los algoritmos de machine learning buscan pred...,Los arboles de decisión pertenecen a los algor...,Consideremos el siguiente ejemplo: Una persona...,¿Se necesitan cientos de modelos para resolver...,NaN
4,H,Python vs R para análisis de datos. A pesar de...,Tipos de aprendizaje en Machine Learning. El M...,Tipos de árboles de decisión y aplicaciones. U...,La combinación de modelos o ensemble es un mod...,En el articulo se analizan cerca de 180 clasif...,XGBoosting vs GBoosting. A pesar de que el alg...


# Exercise 18.1

Analyze the writing patterns of each student

### A continuacion se observan los tokens más comunes para los primeros 5 estudiantes, se consideraron n gramas y un máximo de features de 400.

In [3]:
data.fillna(' ', inplace=True) 
X=data.iloc[:,1]+data.iloc[:,2]+data.iloc[:,3]+data.iloc[:,4]+data.iloc[:,5]+data.iloc[:,6]
vocabs=[]
vocabs2=[]
for i in range(0,data.shape[0]):
    #print(i)
    vect = CountVectorizer(max_features=400,ngram_range=(0, 4))
    X_dtm = vect.fit_transform(X.iloc[i:i+1])
    vocabs.append(vect.vocabulary_)
    vocabs2.append(pd.DataFrame(sorted(vocabs[i].items(),reverse=True)).iloc[:,0])
    
freqs=pd.DataFrame(vocabs2)
freqs.reset_index(inplace=True)
freqs.drop('index',axis=1).head() ##palabras mas comunes para los 5 primeros estudiantes.

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,último,índice,árboles de inferencia,árboles de decisión,árboles de clasificación,árboles de,árboles,árbol de decisión,árbol de,árbol,...,atributo,are,aquellos que,aquellos,and,algoritmos de,algoritmos,aleatorios,al,
1,árboles de decisión,árboles de,árboles,árbol de decisión,árbol de,árbol,xgboost,volúmenes de datos,volúmenes de,volúmenes,...,aprendizaje,ante,algoritmos de,algoritmos,algoritmo es,algoritmo el,algoritmo,aleatoria,al,
2,árboles encontramos,árboles de decisión,árboles de,árboles,árbol,ya que,ya,vez,ventaja,van,...,análisis,anterior,algoritmos comunes,algoritmos,algoritmo,al,además,aclaran que,aclaran,
3,áreas de,áreas,árbol,variables,variable de respuesta observada,variable de respuesta,variable de,variable,uso,uno de,...,alcanzable es,alcanzable,al,ajustar,adquirir,además,adecuada,acuerdo,accuracy,
4,árboles de decisión,árboles de clasificación,árboles de,árboles,árbol para,árbol,ya,xgboost,vs,varios,...,ambos,algunas,algoritmos de,algoritmos,algoritmo,al,además,acuerdo al,acuerdo,


# Exercise 18.2

Evaluate the similarities of the homeworks of the students

tip: https://github.com/orsinium/textdistance

### Se realizó el método TFIDF para calcular la distancia entre los textos de los estudiantes, a continuacion la matriz de distancias.

In [115]:
vect = TfidfVectorizer()
dtm = vect.fit_transform(X)
#print(dtm)

from sklearn.feature_extraction.text import TfidfTransformer
tfidfTran = TfidfTransformer(norm="l2")
tfidfTran.fit(dtm)
#print(tfidfTran.idf_)

tfidf_matrix = tfidfTran.transform(dtm)
#print(tfidf_matrix.toarray())

cos_similarity_matrix = (tfidf_matrix * tfidf_matrix.T).toarray()
print(cos_similarity_matrix)

[[1.         0.27684427 0.27535643 ... 0.28038272 0.2909937  0.30521952]
 [0.27684427 1.         0.40235459 ... 0.41936164 0.42516558 0.44260456]
 [0.27535643 0.40235459 1.         ... 0.42849695 0.44662757 0.43997382]
 ...
 [0.28038272 0.41936164 0.42849695 ... 1.         0.45115084 0.46536208]
 [0.2909937  0.42516558 0.44662757 ... 0.45115084 1.         0.48198886]
 [0.30521952 0.44260456 0.43997382 ... 0.46536208 0.48198886 1.        ]]


# Exercise 18.3

Create a classifier to predict the sex of each student



### Se realizó un Modelo de Maquinas de Vectores de soporte, para predecir el sexo del estudiante con base en el texto asociado. Se obtuvo un f1_score del 80%

In [4]:
y=data['Sexo']

y.shape
X_dtm.shape

(1, 400)

In [5]:
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_dtm, y, test_size=0.33, random_state=23)

In [6]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train,y_train)
print(X_resampled.shape)
print(y_train.shape)

(44, 5892)
(28,)


In [7]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf0 = SVC(kernel='rbf',probability=True)
clf0.fit(X_resampled,y_resampled) 

/Users/juand./anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [10]:
from sklearn.metrics import r2_score, roc_auc_score
y_pred = clf0.predict(X_test)
metrics.f1_score(y_test, y_pred, average='weighted')

0.7912087912087912